In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [6]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/cascade_rcnn_r50_rfp_1x_coco.py')
PREFIX = '../../input/data/'

epoch = 12

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_r50_rfp_1x_coco'

cfg.model.roi_head.bbox_head[0].num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, 'latest.pth')
print(checkpoint_path)
# code/mmdetection_trash/work_dirs/cascade_rcnn_r50_rfp_1x_coco/latest.pth

./work_dirs/cascade_rcnn_r50_rfp_1x_coco/latest.pth


In [7]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [8]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-13 12:26:55,175 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-13 12:26:55,176 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 12:26:55,454 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2021-05-13 12:26:55,483 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 12:26:55,756 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.rfp_conv.weight, layer2.0.rfp_conv.bias, layer3.0.rfp_conv.weight, layer3.0.rfp_conv.bias, layer4.0.rfp_conv.weight, layer4.0.rfp_conv.bias



Use load_from_local loader


In [9]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 9.3 task/s, elapsed: 90s, ETA:     0s

In [10]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'cascade_rcnn_r50_rfp_1x_coco.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.28516388 226.35799 148.28781 253.52736 237...,batch_01_vt/0021.jpg
1,6 0.25753468 453.68307 347.358 511.78647 436.5...,batch_01_vt/0028.jpg
2,1 0.98307794 73.50456 168.79286 374.2088 377.8...,batch_01_vt/0031.jpg
3,2 0.055797294 295.64484 159.72343 352.56006 22...,batch_01_vt/0032.jpg
4,1 0.12343261 387.3617 267.878 504.38818 340.18...,batch_01_vt/0070.jpg
